Twitter Text Report
Chris Kinnison
October 25th, 2022.

   The question that I had been curious in researching more into was what is the public sentiment of the Brown's head coach after these last few games that we have lost. Kevin Stefanski has been the head coach for the past almost 3 years for the Browns. Today, the Browns have a very talented and skillful roster, arguably more so than almost ever before. We objectively  have the best running back in the league as of right now, as well as a great defensive line in Miles Garret and Jadeveon Clowney. Will all of that being said, we are currently on a 4 game loosing streak, with a record of 2 wins and 5 losses. A thing for fans to consider is that even though he is doing not to great as of right now, Kevin did lead us to our first playoff win in 24 years as his first year coaching the Browns.
   Considering all of this, I thought it would be very interesting to see what the fans of the Browns think about their head coach, whether they want to give him a chance to continue, or whether they want a new one completely. Just from my personal experience alone, I know that Browns fans are not the biggest fan of loosing. I have personally heard cries to replace Kevin Stefanski, and I want to know if that is everyone else's opinion as well, or if there is some people going to bat for Kevin and defending him.

In [122]:
import pandas as pd
import json
import urllib
import requests

In [123]:
bt = pd.read_csv('Bearer_Token.txt', header = 0, sep = '\t')

In [124]:
headers = {'Authorization':'Bearer {}'.format(bt['Bearer_Token'].iloc[0])}

To get started, I first developed a query that could lead me to get some answers on the question at hand. I also included certain fields that I though would be helpful in answering my question. An obvious one to include was public metrics. This would let me see the likes and retweet counts on all of the tweets that I would be collecting. This will help give insight on whether a lot of people are agreeing with certain narratives on the head coach or not. A tweet with a high number of likes and retweets would mean that particular statement is agrees upon by many other fans, not just that one individual. Another field that I thought would be interesting to play around with was the created at tweet field. I used this to see what the funs were saying right after the most recent loss, as well as right now.

In [125]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'

In [126]:
query_p = urllib.parse.quote('Kevin Stefanski OR Stefanski Browns OR Stefanski fired')

In [127]:
tweet_fields = 'public_metrics,lang,author_id,created_at'

In [128]:
user_fields = 'username'

In [129]:
expansions = 'entities.mentions.username'

In [130]:
query_url = endpoint + '?query={}&tweet.fields={}&user.fields={}&expansions={}'.format(query_p, tweet_fields, user_fields, expansions)

I then sent for a request from twitter after my query and url had been created.

In [131]:
response = requests.get(query_url, headers = headers)

In [132]:
response.text

'{"data":[{"entities":{"mentions":[{"start":3,"end":16,"username":"TheKennyRoda","id":"60101194"},{"start":23,"end":37,"username":"MGMNorthfield","id":"1860554371"},{"start":38,"end":48,"username":"ReghiRoda","id":"1401205305434857473"}]},"created_at":"2022-10-25T16:37:24.000Z","author_id":"60101194","lang":"en","edit_history_tweet_ids":["1584947244843040769"],"text":"RT @TheKennyRoda: \\uD83D\\uDD25New @MGMNorthfield @ReghiRoda Pod\\uD83D\\uDD25\\n\\uD83C\\uDFC8#Browns (2-5) after 4th loss in a row\\n\\uD83C\\uDFC8Stefanski Stinks\\n\\uD83C\\uDFC8Must Win vs Cincy\\n⚾️Guardia…","id":"1584947244843040769","public_metrics":{"retweet_count":3,"reply_count":0,"like_count":0,"quote_count":0}},{"created_at":"2022-10-25T16:37:12.000Z","author_id":"518130293","lang":"en","edit_history_tweet_ids":["1584947193651154956"],"text":"Who is Stefanski constantly talking to in his head set?  #Browns","id":"1584947193651154956","public_metrics":{"retweet_count":0,"reply_count":0,"like_count":0,"quote_c

Here I create a function in order to get a larger number of texts in my search. Twitters default is 10, and I wanted 300 in order to get a broader view and collect more data and receive more opinions on the matter. After that I add all of it into a single data frame using pandas, and then drop the fields that I felt were not necessary to answering my question.

In [133]:
def twt_recent_search (query, num_pages, headers):
    response_list = []
    next_token = ''
    for i in range(0, num_pages):
        if i > 0:
            this_query = query + "&next_token={}".format(next_token)
        else:
            this_query = query

        this_response = requests.get(query, headers = headers)
        this_response_dict = json.loads(this_response.text)
        response_list.append(this_response_dict)
        next_token = this_response_dict['meta']['next_token']    
    return response_list

In [134]:
my_responses = twt_recent_search(query_url, 30, headers)

In [135]:
len(my_responses)

30

In [136]:
my_responses[2]['data'][0].keys()

dict_keys(['public_metrics', 'id', 'author_id', 'edit_history_tweet_ids', 'text', 'entities', 'lang', 'created_at'])

In [137]:
results_1 = pd.DataFrame.from_records(my_responses)

In [138]:
results_1.head()

,data,includes,meta
0,[{'text': 'RT @TheKennyRoda: 🔥New @MGMNorthfie...,"{'users': [{'id': '60101194', 'name': 'Kenny ""...","{'newest_id': '1584947244843040769', 'oldest_i..."
1,[{'text': 'RT @TheKennyRoda: 🔥New @MGMNorthfie...,"{'users': [{'id': '60101194', 'name': 'Kenny ""...","{'newest_id': '1584947244843040769', 'oldest_i..."
2,"[{'public_metrics': {'retweet_count': 3, 'repl...","{'users': [{'id': '60101194', 'name': 'Kenny ""...","{'newest_id': '1584947244843040769', 'oldest_i..."
3,"[{'lang': 'en', 'created_at': '2022-10-25T16:3...","{'users': [{'id': '60101194', 'name': 'Kenny ""...","{'newest_id': '1584947244843040769', 'oldest_i..."
4,"[{'entities': {'mentions': [{'start': 3, 'end'...","{'users': [{'id': '60101194', 'name': 'Kenny ""...","{'newest_id': '1584947244843040769', 'oldest_i..."


In [139]:
data_list = list(results_1['data'])

In [140]:
data_list_of_dfs = [pd.DataFrame(x) for x in data_list]

In [141]:
data_df = pd.concat(data_list_of_dfs)

In [142]:
pb_df = pd.DataFrame(list(data_df['public_metrics']))

In [143]:
data_df = data_df.join(pb_df)

In [144]:
data_df.head()

,text,created_at,edit_history_tweet_ids,entities,id,public_metrics,lang,author_id,retweet_count,reply_count,like_count,quote_count
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,[1584947244843040769],"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,[1584947244843040769],"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,[1584947244843040769],"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,[1584947244843040769],"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,[1584947244843040769],"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",en,60101194,3,0,0,0


In [149]:
final_df = data_df.drop(['public_metrics', 'edit_history_tweet_ids'], axis = 1)

In [148]:
final_df.head()

,text,created_at,entities,id,lang,author_id,retweet_count,reply_count,like_count,quote_count
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,en,60101194,3,0,0,0
0,RT @TheKennyRoda: 🔥New @MGMNorthfield @ReghiRo...,2022-10-25T16:37:24.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1584947244843040769,en,60101194,3,0,0,0


The quality of the data was pretty good. Unfortunately I had a problem with my function, and it was printing the same 10 tweets 30 different times. That was one thing that held me back on the technical side of things. Another thing was that I could not split the entities up into the data frame but that was not too big of an issue when it came to answering my question. In order for me to get a full range of tweets to see and take into account, I had to constantly change the time so I could look at 10 different tweets and times. Another approach that I could have taken would be to skip the function, and just do what the function does manually. This would have taken much more time so I did not do this hear. With this being said, I was able to collect a lot of data and looking at the different tweets helped me understand what the fans thoughts were on Kevin Stefanski. There was a good amount of tweets that were not very favorable with the current head coach. After reviewing all of the data that was collected, it seems as if the public sentiment is bad toward Kevin, but there are also those who believe that Kevin still deserves a chance at redemption. I read a tweet while researching that actually brought up a good point. Out starting QB is out until game 13, which does not help the team win games at all. That is one thing that I, as well as many others did not think about. I think if I could have read the conversations within the tweets, I would have gotten a better idea of what everyone thinks, but from the likes and retweets of the tweets that I saw, Kevin Stefanski is not a favorable head coach.